In [1]:
import numpy as np
import pandas as pd

In [2]:
X=[]
Y=[]
lines=[]
with open('C://Users//HP//Desktop//ML//Self-Driving-Car//driving_dataset//data.txt') as file:
    lines=file.readlines()
    
for line in lines:
    w=line.split()
    X.append(w[0])
    Y.append(float(w[1]))
        

In [3]:
for i,x in enumerate(X):
    X[i]='C://Users//HP//Desktop//ML//Self-Driving-Car//driving_dataset//'+X[i]
    
X[4]

'C://Users//HP//Desktop//ML//Self-Driving-Car//driving_dataset//4.jpg'

In [4]:
import math

for i,y in enumerate(Y):
    Y[i]=(y*math.pi)/180
    
print(Y[89])

0.024609142453120045


In [5]:
import matplotlib.pyplot as plt

counts,bin_edges=np.histogram(Y,bins=100,density=True)
pdf=counts/sum(counts)
plt.plot(bin_edges[1:],pdf)
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
import keras
import cv2

class Generator(keras.utils.Sequence):
    
    def __init__(self,X,Y,batch_size):
        self.X=X
        self.Y=Y
        self.batch_size=batch_size
        
    def __len__(self):
        return int(np.ceil(len(self.X)/batch_size))
    
    def __getitem__(self,idx):
        batch_x=self.X[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_y=self.Y[idx*self.batch_size : (idx+1)*self.batch_size]
        
        x_vec=[]
        for img in batch_x:
            x_vec.append(cv2.imread(img))
            
        return np.array(x_vec),np.array(batch_y)

In [7]:
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D

width=256
height=455
channels=3
batch_size=256
epochs=30
model=Sequential()

#Conv Layer-1
model.add(Conv2D(50,kernel_size=5,input_shape=(width,height,channels),
                 data_format='channels_last',padding='same'
                 ,activation='relu'))

#Max Pool Layer-1
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#Conv Layer-2
model.add(Conv2D(50,kernel_size=5,activation='relu'))

#Max Pool Layer-2
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#Conv Layer-3
model.add(Conv2D(100,kernel_size=5,activation='relu'))
model.add(Flatten())

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))



In [8]:
# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 455, 50)      3800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 227, 50)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 223, 50)      62550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 111, 50)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 107, 100)      125100    
_________________________________________________________________
flatten (Flatten)            (None, 620600)            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6

In [9]:

X_train=X[0:800]
X_val=X[800:1000]
Y_train=Y[0:800]
Y_val=Y[800:1000]

print("train:",len(X_train),len(Y_train))
print("val:",len(X_val),len(Y_val))

train: 800 800
val: 200 200


In [10]:
from sklearn.metrics import mean_squared_error

z=[0]*len(Y_val)
mean_squared_error(Y_val, z)

6.206562487740743

In [30]:
batch_size = 32

training_batch_generator =Generator(X_train, Y_train, batch_size)
validation_batch_generator =Generator(X_val, Y_val, batch_size)

model.fit(training_batch_generator,
            steps_per_epoch=int(len(X_train)//batch_size),
            epochs = 10,
            verbose=1,
            workers=100,
            validation_data=validation_batch_generator,
            validation_steps=int(len(X_val)//batch_size))

Epoch 1/10
25/25 [==============================] - 207s 8s/step - loss: 102.3934 - mean_absolute_error: 102.3934 - val_loss: 2.9908 - val_mean_absolute_error: 2.9908
Epoch 2/10
25/25 [==============================] - 204s 8s/step - loss: 0.9069 - mean_absolute_error: 0.9069 - val_loss: 2.1242 - val_mean_absolute_error: 2.1242
Epoch 3/10
25/25 [==============================] - 205s 8s/step - loss: 0.8217 - mean_absolute_error: 0.8217 - val_loss: 1.9882 - val_mean_absolute_error: 1.9882
Epoch 4/10
25/25 [==============================] - 204s 8s/step - loss: 0.7702 - mean_absolute_error: 0.7702 - val_loss: 1.9801 - val_mean_absolute_error: 1.9801
Epoch 5/10
25/25 [==============================] - 204s 8s/step - loss: 0.7527 - mean_absolute_error: 0.7527 - val_loss: 1.8902 - val_mean_absolute_error: 1.8902
Epoch 6/10
25/25 [==============================] - 202s 8s/step - loss: 0.7723 - mean_absolute_error: 0.7723 - val_loss: 2.0307 - val_mean_absolute_error: 2.0307
Epoch 7/10
25/25 [

In [11]:
#model.save('m1.h5')
from keras.models import load_model 
model = load_model("m1.h5") 

In [12]:
import numpy as np
import cv2

def rotate_image(img, smoothed_angle, degrees):
    rows,cols,_ = img.shape
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    
    return dst, smoothed_angle

In [14]:
X_val=X[35000:]
Y_val=Y[35000:]
i=0
import os
wheel=cv2.imread('C://Users//HP//Desktop//ML//Self-Driving-Car//wheel-4.jpg')
smoothed_angle=0
smoothed_angle_actual=0
for x in X_val:
    img=cv2.imread(x)
    cv2.imshow('frame',img)
    
    rad=model.predict(np.array([img]))[0][0]
    deg=(rad*180)/(math.pi)
    
    w,smoothed_angle=rotate_image(wheel,smoothed_angle,deg)
    cv2.imshow('Prediction',w)
    
    deg2=(Y_val[i]*180)/(math.pi)
    
#     s="Prediction: "+str(deg)+" Actual: "+str(deg2)+"\n"
#     s=s.encode()
#     os.write(1,s)
    w2,smoothed_angle_actual=rotate_image(wheel,smoothed_angle_actual,deg2)
    cv2.imshow('Actual',w2)
    
    i+=1;
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

ZeroDivisionError: float division by zero

In [14]:
wheel.shape

(400, 400, 3)